In [12]:
import numpy as np
import sys
import os
from pathlib import Path
import itertools



dir_path = Path.cwd()

print(dir_path)
sys.path.insert(0, dir_path)

/home/nikolaos/Workspaces/AML/Assignment_1_Programming/code


In [66]:
dataPath = os.path.join(dir_path, '../data')
letters = 'abcdefghijklmnopqrstuvwxyz'
# Load decode Input
decInputFile = os.path.join(dataPath, 'decode_input.txt')
with open(decInputFile, 'r') as f:
    inData = f.read().splitlines()
    inX = np.asarray(inData[0:12800], dtype = np.float)     # input is 100 letters @ 128 = 12800, or 0-12799. NOTE: Recall Python slice is [).
    inW = np.asarray(inData[12800:16128], dtype = np.float) # wieths are 26 * 128 = 3328, or 12800-16127
    inT = np.asarray(inData[16128:], dtype = np.float)      # The remaing are the transition probs.
print("Done reading! Sizes are X: {}, W:{},T: {}".format(inX.shape, inW.shape, inT.shape))

Done reading! Sizes are X: (12800,), W:(3328,),T: (676,)


In [68]:
# Reshape arrays
# Reshape T to 26x26
inT = np.reshape(inT,(26,26)) 
inX = np.reshape(inX, (-1,128))
inW = np.reshape(inW, (128,-1))
print("Done reshaping! Sizes now are X: {}, W:{},T: {}".format(inX.shape, inW.shape, inT.shape))

Done reshaping! Sizes now are X: (100, 128), W:(128, 26),T: (26, 26)


In [118]:
# Max-Sum algorithm!
def crf_decoder(seq, w, t):
    # Init
    seqLen  = seq.shape[0]
    dataLen = w.shape[1]
    cMat    = np.zeros((seqLen, dataLen))
    bMat    = np.matmul(seq, w) # this gives the probability of observing each i standalone
    maxIdx  = np.zeros((seqLen, dataLen), dtype = np.int)
    decSeq  = []
    ci = np.zeros((26,26))
    
    # Get step 0 estimates
    step0    = np.matmul(seq[0,:], w)
    c1 = np.multiply(step0,t)      # the previous node potential x edje potenital for all i->j pairs
    maxIdx[0,:] = np.argmax(c1, axis = 0) #1x26 the max elem indexes
    c1 = c1[maxIdx[0,:],range(c1.shape[1])] #1x26 select the max element, as indexed by maxIdx, for every column
    cMat[0,:] = c1
    print("Input shapes: sequence: {}, weights: {}, T matrix:{}".format(seq.shape,w.shape,t.shape))
    
    # Perform a pass over steps 1-99 or 2-100 in 1-index notation!
    for i in range(1,seqLen):
        fi  = bMat[i,:]              # get likelihood of i observation being any letter as standalone 
        cYi = cMat[i-1,:]            # i-1 step's potential msg to this node i
        cYj = np.multiply(fi, t)     # mul the likelihood of each letter with all transition probs.
        maxIdx[i,:] = np.argmax(cYj, axis = 0) #1x26 the max elem indexes
        cYj = cYj[maxIdx[i,:],range(cYj.shape[1])] #1x26 select the max element, as indexed by maxIdx, for every column
        cYj = np.multiply(cYj, cYi)  # multiply current cYj = g_yj->yi* fj with pervious node's(i-1) msgs to yield the msg node i will wend to i+1
        if i == 1:
            print(cYj, cYj.shape, fi.shape)
        #mostProbState = np.matmul(res[i-1,:], t)
    #TODO: Perform the backward pass to get most prob sequence
        
    #labels = np.argmax(s, axis = 1)
    #result = [letters[i] for i in labels]
    return decSeq
    
crf_decoder(inX, inW, inT)

Input shapes: sequence: (100, 128), weights: (128, 26), T matrix:(26, 26)
[7.78442958e-03 1.52115672e-02 1.20710706e-02 4.69876271e-03
 5.34670451e-01 1.14755986e-02 5.64371856e-02 3.31816960e-02
 4.00406494e-03 1.53669183e-01 4.88480308e-01 4.30666478e-05
 3.11189330e-01 7.07851358e-02 6.51891449e-02 1.29705870e-01
 1.68724845e-03 4.50357985e-01 6.23706302e-02 3.08509408e-03
 2.69362883e-02 3.39592362e-01 2.16999483e-01 3.78132364e-02
 1.30214979e-02 2.69906264e-01] (26,) (26,)


[]

In [70]:
print(result)

['f', 'p', 'd', 'k', 'j', 'o', 'i', 'u', 'y', 't', 'g', 'j', 'r', 'a', 'v', 'm', 'j', 'm', 's', 'u', 'n', 'h', 'y', 'n', 'p', 'w', 'h', 'm', 'e', 'v', 'j', 'k', 'z', 'p', 'n', 'b', 'i', 'r', 'b', 'z', 'g', 'v', 'm', 'k', 'd', 'd', 'i', 'e', 'x', 'r', 'f', 'e', 'y', 'm', 'm', 'x', 'c', 'o', 'm', 'r', 'x', 'x', 'x', 'n', 'c', 's', 'k', 'v', 'n', 'r', 'u', 'a', 'y', 'o', 'k', 'y', 'r', 'p', 'h', 'l', 'b', 'j', 't', 'w', 'l', 'u', 'w', 'p', 'q', 'a', 'i', 'g', 'r', 'f', 'v', 'u', 'd', 'z', 'a', 'p']
